### Please take care to regularly create snapshots (File, Save and Checkpoint). This way we can go back to this backup in case something goes bad. 


# Contents
1. [Genome assembly](#De novo genome assembly)
    1. [De Bruijn graphs](#De Bruijn graphs)
    + [Overlap-Layout-Consensus](#OLC)


# 1. De novo genome assembly <a name="De novo genome assembly"></a>


The decrease in cost and increase in throughput of sequencing technologies has allowed the sequencing of multiple organisms in the recent years. De novo genome assembly consists in joining the sequencing reads together to assemble the original genome. You can think of this as solving a puzzle. The fundamental feature behind genome assembly is finding the overlaps between the reads. The way to represent these overlaps is using graphs. 
There are many advantages of having an assembled genome instead of the sequencing reads (even when mapped). The most obvious is the  sequencing of new species for which there is no reference genome available. But also, to find new regions that are not present in a reference genome, investigate the structural variation or find new genes. In this assignment, you will use both short read (Illumina) and long read (Oxford Nanopore) sequencing data to assemble some part of the human genome. Since assembling an entire human genome or even one chromosome would take too much computational time and memory we will limit ourselves to the BRCA1 and BRCA2 loci.  
Before starting a quick reminder of some terms that are essential to genome assembly:
 - **Read:** Any sequence that comes out of the sequencer. Short reads are between 100-300 base pairs (bp) and long reads are > 1000 bp, reaching even hundreds of thousands of base pairs. 
 - **Coverage:** Sequencing depth; Average number of reads per base in the genome. 
 - **_k_-mer:** Sub-sequence of length _k_.
 - **Contig:** Assembled sequence without any gaps. 
 - **N50:** Size of the contig which, together with all larger contigs, amounts for over 50% of the total assembly. Gives an idea about the contiguity of an assembly.

## A) Assembling short reads: De Bruijn graphs <a name="De Bruijn graphs"></a>
-------------------------------------------------------------------
To build a De Bruijn graph (DBG), the set of reads is divided into _k-mers_ (sub-sequences of length _k_). For example for a read of length 12 there are 4 possible k-mers of _k_ = 9:

Read:

ATCAATGGCATC

9-mers:

ATCAATGGC

&nbsp;&nbsp;TCAATGGCA

&nbsp;&nbsp;&nbsp;&nbsp;CAATGGCAT
  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;AATGGCATC
   
   
The nodes of a DBG represent these k-mers and the edges between the nodes represent the overlaps between them. These overlaps are exact and have a length _k_ - 1. Once the DBG is formed, it can be traversed using an _Eulerian_ path, which means that all edges need to be traversed once. In the following figure a 3-mer DBG is formed from reads of length 4.

![De Bruijn graphs](http://www.nature.com/nrg/journal/v14/n5/images/nrg3433-f1.jpg)
Berger et al, Computational solutions for omics data. _Nature Reviews Genetics 14, 333–346 (2013)_

The main enemies of De Bruijn Graphs are sequencing errors, since they will create false edges in the graph that will need to be traversed. Also, repeats in the genome longer than the length of the sequencing read will create bubbles in the graph.

You will now assemble the 2 BRCA loci with the Illumina WGS reads that you used already in previous assignments (datafiles/na12878_wgs_brcagenes-1.fastq **and** datafiles/na12878_wgs_brcagenes-2.fastq). __Velvet__ is a widely used DBG-based assembly tool (https://www.ebi.ac.uk/~zerbino/velvet/Manual.pdf , https://en.wikipedia.org/wiki/Velvet_assembler). It uses a two-step approach: 1) _velveth_ "hashes" the reads, i.e. indexes them to find which _k_-mers occur in which reads; 2) _velvetg_ creates a DBG, simplifies it (removes errors) and extracts the contigs.



Our paired-end dataset contains 54350 read pairs that add up to 16.4 mega-base pairs (Mbp). The total size of the loci we aim to assemble is 400 kilo-base pairs (Kbp).

<span style="color:purple">[Q]: What is the average sequencing coverage over this region?</span>

[A]:

<span style="color:purple">[Q]: How many contigs do you expect to have as a result of the assembly?</span>

[A]:

<span style="color:green">[DO]: Run _velveth_ to create the hash table on the fastq files. First execute just _velveth_ to read the different parameters you can provide. You will need to provide: 1) an output directory; 2) a hash length ( = _k_-mer length. We will use _k_ = 25); 3) some facts about the source of the data (we have short paired reads in separate FASTQ files); 4) the files two files with the reads. </span>

<span style="color:green">[DO]: Now run _velvetg_ to create the De Bruijn Graph and subsequently the contigs. Again, first execute just _velvetg_ to read the options. You will need to provide: 1) the directory where the hash table was formed; 2) the expected coverage; 3) the insert length since we have paired end reads (insert length = 550) 4) Set the minimum contig length to 200.  </span>

The last line of the output that _velveth_ prints out already gives some useful information about the assembly process.

<span style="color:purple">[Q]: What percentage of the reads was not used?</span>

[A]:

<span style="color:green">[DO]: The output file is created under $outputDir/contigs.fa . This is a FASTA file. To obtain the metrics of the assembly and answer the following questions, you can either explore the file yourself using Python or use an existing tool by executing:

assembly-stats $outputDir/contigs.fa  </span>

<span style="color:purple">[Q]: How many bases have been assembled in total? Into how many contigs? What is the size of the largest contig?</span> 

[A]: 

<span style="color:purple">[Q]: What is the N50? How many contigs are larger than this N50? And smaller?</span>

[A]:

<span style="color:purple">[Q]: An N is introduced in a contig when there is evidence that two nodes can be connected but not about the specific nucleotide. How many Ns do we have in our assembly? In your opinion, what is the source of this Ns? </span>

[A]:

<span style="color:green">[DO]: Now run velvet again but using a _k_-mer length of 31. You will need to execute _velveth_ and _velvetg_ again.  Answer the same questions with this new assembly. </span>

<span style="color:purple">[Q]: How many bases have been assembled in total? Into how many contigs? What is the size of the largest contig?</span> 

[A]: 

<span style="color:purple">[Q]: What is the N50? How many contigs are larger than this N50? And smaller?</span>

[A]: 

<span style="color:purple">[Q]: An N is introduced in a contig when there is evidence that two nodes can be connected but not about the specific nucleotide. How many Ns do we have in our assembly? In your opinion, what is the source of this Ns? </span>

[A]: 

<span style="color:purple">[Q]: How did the k-mer length affect all these metrics? Which assembly do you think is better and why? </span>

[A]: 

## B) Assembling long reads: Overlap - Layout - Consensus <a name="OLC"></a>
-------------------------------------------------------------------
In the latest years new sequencing technologies allow to sequence longer fragments of DNA, generating reads of tens and even hundreds of kilo base pairs (Kbp). However, the per base error rate is higher that with Illumina short read sequencing. As mentioned above, De Bruijn graphs are very sensitive to sequencing errors, therefore they are not applicable with this type of data. Nonetheless, the increased read length allows to span repeats and find longer unique overlaps. There a more straightfoward algorithm can be used. Overlap-Layout-Consensus (OLC) works in three steps:
1. **Overlap:**Find the overlapping reads. This is a computationally expensive step, given the large number of pairwise comparisons needed. The overlap graph is then constructed. In this case, nodes directly represent the reads and edges represent the overlaps between them (with no fixed length). 
+ **Layout**: First simplify the graph. Reads (nodes) that are completely contained in other reads are removed. Redundant edges are also remove using _transitive reduction_:If a > c and a > b > c, then remove a > c. Finally, the simplified graph is traversed using a _Hamiltonian_ path, where all nodes need to be visited once. Contigs are formed in this step.
+ **Consensus:** Find the most likely nucleotide for each position of each contig using multiple sequence alignment. 

![OLC graphs](https://www.researchgate.net/profile/Christina_Toft/publication/26266221/figure/fig2/AS:216492458156039@1428627232413/Overlap-layout-consensus-genome-assembly-algorithm-Reads-are-provided-to-the-algorithm.png)
Commins et al, Computational Biology Methods and Their Application to the Comparative Genomics of Endocellular Symbiotic Bacteria of Insects. _Biological Procedures Online 11.1, (2009)_

We will use the same BRCA1/2 loci but now with MinION reads. We have in total 2880 reads that sum up to 13.1 Mbp. 

<span style="color:purple">[Q]: What is the average read length in this dataset? What is an estimation of the coverage for the total region we aim to assemble? </span>

[A]:

We will now use Miniasm, an ultra-fast assembler (https://github.com/lh3/miniasm). It avoids certain computationally expensive steps (uses approximate overlaps, consensus is not directly performed). It also runs in two steps, first **minimap** finds the overlaps between the reads and then **miniasm** build the graph and finds the path to transverse it. 

<span style="color:green">[DO]: First run minimap to find the overlaps between the reads.   </span>

<span style="color:green">minimap -x ava10k datafiles/COO5/na12878_ont_brcagenes.fastq datafiles/COO5/na12878_ont_brcagenes.fastq > datafiles/COO5/brca.ont.paf   </span>

<span style="color:green">(-x ava10k tells minimap that we have ONT data).   </span>


<span style="color:purple">[Q]: Why do you have to input the FASTQ file with the reads twice? </span>

[A]:

<span style="color:green">[DO]: Now we need to run Miniasm to build the layout graph and find contigs.   </span>

<span style="color:green">miniasm -f datafiles/COO5/na12878_ont_brcagenes.fastq datafiles/COO5/brca.ont.paf | awk '/^S/{print ">"\$2"\\n"\$3}' > datafiles/COO5/brca.ont.fasta  </span>

<span style="color:green">( This tools outputs the assembly in a different format than FASTA, so an extra step after the | pipe is needed).   </span>


Once it is done, lets take a look at the assembly. 

<span style="color:green">[DO]: Explore the resulting contig file. This time is under datafiles/COO5/brca.ont.fasta. Again, you can use your own Python code or run assembly-stats on the contigs file </span>

<span style="color:green"> assembly-stats datafiles/COO5/brca.ont.fasta </span>

<span style="color:purple">[Q]: How many bases have been assembled in total with the long reads? Into how many contigs? What are the sizes of these contigs?</span> 

[A]:

<span style="color:purple">[Q]: What is the N50? How relevant do you think the N50 is in this case?</span>

[A]:

<span style="color:purple">[Q]: Why do you think we do not have Ns in this assembly?</span>

[A]:

As you can see, the long reads that are nowadays available generate much more contiguous assemblies. Repeat structures and structural variation otherwise missed by short read sequencing are now being revealed. Maybe soon it will allow the finishing of the human genome (right now it contains more than 800 gaps, after decades and billions of \$ spent). For a short overview of the current state of genome assembly, if you are interested, read this recently published editorial by Adam Phillippy: http://genome.cshlp.org/content/27/5/xi.short 